### Download the data

In [1]:
!wget https://raw.githubusercontent.com/suvigyajain0101/CaseStudies/main/AdverseEventClassification/Data/AE_Data.csv

--2022-09-21 22:10:18--  https://raw.githubusercontent.com/suvigyajain0101/CaseStudies/main/AdverseEventClassification/Data/AE_Data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5998096 (5.7M) [text/plain]
Saving to: ‘AE_Data.csv’

AE_Data.csv         100%[===================>]   5.72M  --.-KB/s    in 0.08s   

2022-09-21 22:10:19 (71.7 MB/s) - ‘AE_Data.csv’ saved [5998096/5998096]



In [3]:
! pip install --quiet bertopic

     |████████████████████████████████| 90 kB 3.9 MB/s 
     |████████████████████████████████| 636 kB 39.2 MB/s 
     |████████████████████████████████| 5.2 MB 46.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 85 kB 4.7 MB/s 
     |████████████████████████████████| 88 kB 980 kB/s 
     |████████████████████████████████| 4.9 MB 36.4 MB/s 
     |████████████████████████████████| 1.3 MB 37.3 MB/s 
     |████████████████████████████████| 120 kB 57.1 MB/s 
     |████████████████████████████████| 6.6 MB 36.2 MB/s 
     |████████████████████████████████| 1.1 MB 48.6 MB/s 


In [6]:
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

In [7]:
df = pd.read_csv('/content/AE_Data.csv')
df.head()

,title,abstract,label
0,antimicrobial impacts of essential oils on foo...,the antimicrobial activity of twelve essential...,0
1,purification and characterization of a cystein...,antimicrobial peptide (amp) crustin is a type ...,0
2,telavancin activity tested against gram-positi...,objectives: to reassess the activity of telava...,0
3,the in vitro antimicrobial activity of cymbopo...,background: it is well known that cymbopogon (...,0
4,screening currency notes for microbial pathoge...,fomites are a well-known source of microbial i...,0


## Out of the Box

In [9]:
# we add this to remove stopwords
vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words="english")

model = BERTopic(
    vectorizer_model=vectorizer_model,
    language='english', calculate_probabilities=True,
    verbose=True
)


In [10]:
topics, probs = model.fit_transform(df['abstract'])

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

Batches:   0%|          | 0/130 [00:00<?, ?it/s]

2022-09-21 22:27:33,893 - BERTopic - Transformed documents to Embeddings
2022-09-21 22:28:13,797 - BERTopic - Reduced dimensionality
2022-09-21 22:28:15,686 - BERTopic - Clustered reduced embeddings


In [11]:
freq = model.get_topic_info()
freq.head(10)

,Topic,Count,Name
0,-1,1119,-1_strains_bacteria_activity_andotherspecies
1,0,154,0_biofilm_oral_plaque_periodontal
2,1,137,1_compounds_synthesized_derivatives_antibacterial
3,2,129,2____
4,3,126,3_extracts_extract_activity_plants
5,4,120,4_naocl_antimicrobial_microorganisms_agar
6,5,106,5_ciprofloxacin_active_mgl_fluoroquinolones
7,6,96,6_utis_mic90s_urinary_urinary tract
8,7,84,7_ceftobiprole_ceftaroline_ceftazidime_cephalo...
9,8,80,8_oil_essential_oils_essential oil
